# **FORECAST CREDITOS GENERALES**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo',
 'df_financiamientos_2024_tulancingo']

# **Forecast créditos generales (cantidad)**

### BASE

In [14]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [15]:
# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y contar el número de registros únicos
    total_creditos_df = df.groupby(['mes']).size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'num_registros']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_conteo_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_conteo_creditos_pachuca

,año,mes,num_registros
0,2019,1,16
1,2019,2,28
2,2019,3,29
3,2019,4,33
4,2019,5,40
...,...,...,...
62,2024,3,17
63,2024,4,30
64,2024,5,30
65,2024,6,33


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [17]:
from sklearn.metrics import r2_score
import numpy as np

# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.2999237873798627
Error cuadrático medio (MSE): 32.889294723501145
Error absoluto medio (MAE): 4.821457822759579
Raíz del error cuadrático medio (RMSE): 5.734918894239145


In [18]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
nuevas_fechas.head()

total_conteo_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_conteo_pachuca

,año,mes,num_registros
0,2019,1,16
1,2019,2,28
2,2019,3,29
3,2019,4,33
4,2019,5,40
...,...,...,...
92,2026,8,24
93,2026,9,25
94,2026,10,26
95,2026,11,26


In [19]:
# Asegurarse de que la columna 'año' es de tipo entero
#total_conteo_pachuca['año'] = total_conteo_pachuca['año'].astype(int)

total_conteo_pachuca = total_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_conteo_pachuca['modalidad'] = 'Créditos (todos)'
#total_conteo_pachuca = total_conteo_pachuca.groupby('año').agg({'num_registros': 'sum', 'modalidad': 'first'}).reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales
# Formatear la columna 'montos' con la función personalizada
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].map(format_number)
total_conteo_pachuca = total_conteo_pachuca.iloc[1:].reset_index(drop=True)

total_conteo_pachuca

,año,num_registros,modalidad
0,2025,292,Créditos (todos)
1,2026,273,Créditos (todos)
2,2019,406,Créditos (todos)
3,2020,389,Créditos (todos)
4,2021,371,Créditos (todos)
5,2022,316,Créditos (todos)
6,2023,312,Créditos (todos)
7,2024,197,Créditos (todos)


In [20]:
# Valores conocidos
registros_2022 = 316
registros_2023 = 312

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))


Estimación de registros en 2024: 308


In [21]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].replace({',': ''}, regex=True).astype(int)

# Ahora intenta realizar la sustitución
total_conteo_pachuca.loc[total_conteo_pachuca['año'] == "2024", 'num_registros'] = 308

# Verifica el cambio en el DataFrame
print(total_conteo_pachuca)


    año  num_registros         modalidad
0  2025            292  Créditos (todos)
1  2026            273  Créditos (todos)
2  2019            406  Créditos (todos)
3  2020            389  Créditos (todos)
4  2021            371  Créditos (todos)
5  2022            316  Créditos (todos)
6  2023            312  Créditos (todos)
7  2024            308  Créditos (todos)


### Modalidad 1: Nueva

In [22]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y contar el número de registros únicos por modalidad en cada año
    conteo_creditos_año = df.groupby('modalidad').size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    conteo_creditos_año['año'] = año
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,num_registros,año
0,1,133,2019
1,2,81,2019
2,3,190,2019
3,4,2,2019
4,1,98,2020
5,2,90,2020
6,3,195,2020
7,4,6,2020
8,1,80,2021
9,2,67,2021


In [23]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y contar el número de registros
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año']).size().reset_index(name='num_registros')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes   1   2   3   4   5   6   7   8   9   10  11  12
 año                                                 
 2019   7   8   7  10  17  15   8  11   8  19  10  13
 2020   9   7  14   3  11   8  11   7   8   1   6  13
 2021   5   6   8   6   2   2  11  13  10   3   0  14
 2022  10   8  10   6   7   8   9   8   8  10   3   8
 2023   5   8   8   2   5   7   3   7   7  12   7   5
 2024   4   5   4   3  13  11   9   0   0   0   0   0,
 mes   1   2   3   4   5   6   7   8   9   10  11  12
 año                                                 
 2019   7  13  15  11  15  17  19  17  17  23  21  15
 2020  14  12  19   3  10  27  17  13  18  19  18  25
 2021  10  17  21  19  19  22  18  12  20  19  13  24
 2022   7  11  15   9  12  13  15  13  17  16  13  20
 2023   8   9  14  15  18  14  17  16  16  21  15  17
 2024   8  13   7  16   5  10  18   0   0   0   0   0)

In [24]:
X = df_modalidad_1[['año', 'mes']]
y = df_modalidad_1['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [25]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024,6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 


# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([df_modalidad_1, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca['modalidad'] = 'Nueva'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo_pachuca['num_registros'] = nueva_conteo_pachuca['num_registros'].map(format_number)
nueva_conteo_pachuca

,año,num_registros,modalidad
0,2019,133,Nueva
1,2020,98,Nueva
2,2021,80,Nueva
3,2022,95,Nueva
4,2023,76,Nueva
5,2024,89,Nueva
6,2025,66,Nueva
7,2026,59,Nueva


In [26]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [27]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 


# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,num_registros,modalidad
0,2019,190,Usada
1,2020,195,Usada
2,2021,214,Usada
3,2022,161,Usada
4,2023,180,Usada
5,2024,174,Usada
6,2025,164,Usada
7,2026,157,Usada


In [28]:
conteo_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [29]:
# Convertir 'num_registros' a enteros y eliminar comas
conteo_pachuca['num_registros'] = conteo_pachuca['num_registros'].apply(lambda x: int(str(x).replace(',', '')))

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año  num_registros         modalidad
2  2019            406  Créditos (todos)
0  2019            133             Nueva
0  2019            190             Usada
1  2020             98             Nueva
1  2020            195             Usada
3  2020            389  Créditos (todos)
4  2021            371  Créditos (todos)
2  2021             80             Nueva
2  2021            214             Usada
3  2022            161             Usada
3  2022             95             Nueva
5  2022            316  Créditos (todos)
4  2023             76             Nueva
6  2023            312  Créditos (todos)
4  2023            180             Usada
7  2024            308  Créditos (todos)
5  2024             89             Nueva
5  2024            174             Usada
0  2025            292  Créditos (todos)
6  2025             66             Nueva
6  2025            164             Usada
1  2026            273  Créditos (todos)
7  2026             59             Nueva
7  2026         

In [30]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['num_registros'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['num_registros'],  # Añadir el texto de los valores
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    #yaxis_title="Número de Registros",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Configurar el eje Y para mostrar los valores completos en miles (sin 'K')
fig.update_yaxes(tickformat=",")  # ',' para formato de miles sin abreviación


# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_cantidad', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [TOTAL]**

In [20]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).sum().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_suma_creditos_pachuca

,año,mes,montos
0,2019,1,11787572.19
1,2019,2,16520649.45
2,2019,3,14752147.62
3,2019,4,14434520.60
4,2019,5,27661094.33
...,...,...,...
62,2024,3,6374127.97
63,2024,4,30724747.58
64,2024,5,23488340.60
65,2024,6,26764458.63


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_suma_creditos_pachuca[['año', 'mes']]
y = total_suma_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [23]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
 #                                 'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_suma_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)

total_todos_pachuca = total_todos_pachuca.iloc[1:].reset_index(drop=True)

total_todos_pachuca

,año,montos,modalidad
0,2025,308092811,Créditos (todos)
1,2026,317800851,Créditos (todos)
2,2019,259936080,Créditos (todos)
3,2020,267516703,Créditos (todos)
4,2021,289356367,Créditos (todos)
5,2022,264352549,Créditos (todos)
6,2023,281908842,Créditos (todos)
7,2024,157828168,Créditos (todos)


In [24]:
# Valores conocidos
registros_2022 = 264352549
registros_2023 = 281908842

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))

Estimación de registros en 2024: 300631091


In [25]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Ahora intenta realizar la sustitución
total_todos_pachuca.loc[total_todos_pachuca['año'] == "2024", 'montos'] = 300631091


# Verifica el cambio en el DataFrame
print(total_todos_pachuca)


    año     montos         modalidad
0  2025  308092811  Créditos (todos)
1  2026  317800851  Créditos (todos)
2  2019  259936080  Créditos (todos)
3  2020  267516703  Créditos (todos)
4  2021  289356367  Créditos (todos)
5  2022  264352549  Créditos (todos)
6  2023  281908842  Créditos (todos)
7  2024  300631091  Créditos (todos)


In [36]:
df_financiamientos_2024_queretaro.shape

(5435, 17)

In [26]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].sum().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, suma_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,montos,año
0,1,1.105762e+08,2019
1,2,2.497828e+06,2019
2,3,1.447695e+08,2019
3,4,2.092558e+06,2019
4,1,1.089469e+08,2020
5,2,1.497066e+06,2020
6,3,1.517374e+08,2020
7,4,5.335386e+06,2020
8,1,8.355213e+07,2021
9,2,1.007060e+06,2021


In [27]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].sum().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes            1           2            3           4            5   \
 año                                                                   
 2019   6823749.78  6745626.45   4913661.59  7475032.19  16318059.94   
 2020  11370491.20  5909066.23   9182592.29        0.00   9328727.31   
 2021   7184383.07  5248478.57  12919404.71  5445424.84   1445739.82   
 2022   7877320.49  6929291.21  13921627.06  3893531.90   8182938.77   
 2023   7354088.19  6905871.33   8269766.03  1873905.47   4753393.97   
 2024   4267751.01  8341636.76    727032.00  2663807.31  13386632.91   
 
 mes            6            7            8            9            10  \
 año                                                                     
 2019  11526785.86   4782263.36   9244387.91   6219248.36  16991263.15   
 2020   7645418.56  11127389.87  14130064.40  12628030.71    980042.70   
 2021   1505444.92   8811877.70  12520572.47   7993918.21   2227127.05   
 2022  10423027.08   8040405.67   6369459.32   80002

In [28]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]

In [29]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo_pachuca = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].sum().reset_index()

# Mostrar el DataFrame resultante
print(nueva_conteo_pachuca)

    año modalidad        montos
0  2019     Nueva  1.105762e+08
1  2020     Nueva  1.089469e+08
2  2021     Nueva  8.355213e+07
3  2022     Nueva  9.345584e+07
4  2023     Nueva  8.584406e+07
5  2024     Nueva  1.005735e+08
6  2025     Nueva  8.195999e+07
7  2026     Nueva  7.750802e+07


In [30]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)


LinearRegression()

In [31]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
#def format_number(x):
 #   return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
#mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,montos,modalidad
0,2019,1.447695e+08,Usada
1,2020,1.517374e+08,Usada
2,2021,1.827092e+08,Usada
3,2022,1.561768e+08,Usada
4,2023,1.761740e+08,Usada
5,2024,1.762622e+08,Usada
6,2025,1.796029e+08,Usada
7,2026,1.841738e+08,Usada


In [32]:
conteo_pachuca = pd.concat([
    total_todos_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [33]:
conteo_pachuca

,año,montos,modalidad
0,2025,3.080928e+08,Créditos (todos)
1,2026,3.178009e+08,Créditos (todos)
2,2019,2.599361e+08,Créditos (todos)
3,2020,2.675167e+08,Créditos (todos)
4,2021,2.893564e+08,Créditos (todos)
5,2022,2.643525e+08,Créditos (todos)
6,2023,2.819088e+08,Créditos (todos)
7,2024,3.006311e+08,Créditos (todos)
0,2019,1.105762e+08,Nueva
1,2020,1.089469e+08,Nueva


In [34]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año     montos         modalidad
2  2019  259936080  Créditos (todos)
0  2019  110576187             Nueva
0  2019  144769506             Usada
1  2020  108946882             Nueva
1  2020  151737369             Usada
3  2020  267516703  Créditos (todos)
4  2021  289356367  Créditos (todos)
2  2021   83552133             Nueva
2  2021  182709207             Usada
3  2022  156176801             Usada
3  2022   93455838             Nueva
5  2022  264352549  Créditos (todos)
4  2023   85844062             Nueva
6  2023  281908842  Créditos (todos)
4  2023  176174027             Usada
7  2024  300631091  Créditos (todos)
5  2024  100573528             Nueva
5  2024  176262204             Usada
0  2025  308092811  Créditos (todos)
6  2025   81959988             Nueva
6  2025  179602920             Usada
1  2026  317800851  Créditos (todos)
7  2026   77508018             Nueva
7  2026  184173756             Usada


In [35]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto total anual",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)

# Configurar el eje Y utilizando el formateo personalizado
tickvals = [1_000, 1_000_000, 1_000_000_000,5_000_000_000,10_000_000_000,15_000_000_000,20_000_000_000,25_000_000_000, 30_000_000_000]  # Valores hasta 30 mil millones
ticktext = [formatear_numero(val) for val in tickvals]  # Formatea cada valor en palabras

fig.update_yaxes(
    tickvals=tickvals, 
    ticktext=ticktext
)

# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_total', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [PROMEDIO]**

In [32]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).mean().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos = pd.concat(dfs, ignore_index=True)
total_suma_creditos

,año,mes,montos
0,2019,1,7.367233e+05
1,2019,2,5.900232e+05
2,2019,3,5.086947e+05
3,2019,4,4.374097e+05
4,2019,5,6.915274e+05
...,...,...,...
62,2024,3,3.749487e+05
63,2024,4,1.024158e+06
64,2024,5,7.829447e+05
65,2024,6,8.110442e+05


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Preparar datos para el modelo
X = total_suma_creditos[['año', 'mes']]
y = total_suma_creditos['montos']
# Normalizar los datos
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [34]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            #'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026
nuevas_fechas = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_suma_creditos, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)

# Eliminar el primer registro 
#total_todos_pachuca = total_todos_pachuca.iloc[1:]
total_todos_pachuca = total_todos_pachuca.iloc[:-1]
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


total_todos_pachuca

c:\Users\yoe11\anaconda3\envs\newst\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



,año,montos,modalidad
0,2024,634105096,Créditos (todos)
1,2025,634417577,Créditos (todos)
2,2026,634730058,Créditos (todos)
3,2019,638282,Créditos (todos)
4,2020,673959,Créditos (todos)
5,2021,772304,Créditos (todos)
6,2022,833997,Créditos (todos)
7,2023,905381,Créditos (todos)


In [19]:
# Valores conocidos
registros_2022 = 833997
registros_2023 = 905381

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024, 2025, y 2026 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)
estimacion_2025 = estimacion_2024 * (1 + tasa_cambio)
estimacion_2026 = estimacion_2025 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))
print("Estimación de registros en 2025:", round(estimacion_2025))
print("Estimación de registros en 2026:", round(estimacion_2026))

Estimación de registros en 2024: 982875
Estimación de registros en 2025: 1067002
Estimación de registros en 2026: 1158329


In [20]:
# Remover comas y convertir la columna 'montos' a enteros
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Ahora intenta realizar la sustitución
total_todos_pachuca.loc[total_todos_pachuca['año'] == "2024", 'montos'] = 982875
total_todos_pachuca.loc[total_todos_pachuca['año'] == 2025, 'montos'] = 1067002
total_todos_pachuca.loc[total_todos_pachuca['año'] == 2026, 'montos'] = 1158329
total_todos = total_todos_pachuca.copy()
# Verifica el cambio en el DataFrame
print(total_todos)


    año   montos         modalidad
0  2024  1022550  Créditos (todos)
1  2025  1067002  Créditos (todos)
2  2026  1158329  Créditos (todos)
3  2019   638282  Créditos (todos)
4  2020   673959  Créditos (todos)
5  2021   772304  Créditos (todos)
6  2022   833997  Créditos (todos)
7  2023   905381  Créditos (todos)


In [5]:
# Valores conocidos
registros_2024 = 634105096  
registros_2025 = 634417577  
registros_2026 = 634730058  

# Calcula la tasa de cambio porcentual de 2024 a 2025
tasa_cambio_2024_2025 = (registros_2025 - registros_2024) / registros_2024

# Calcula la tasa de cambio porcentual de 2025 a 2026
tasa_cambio_2025_2026 = (registros_2026 - registros_2025) / registros_2025

# Calcula la tasa de cambio porcentual promedio de 2024 a 2026
tasa_cambio_promedio = (tasa_cambio_2024_2025 + tasa_cambio_2025_2026) / 2

# Aplica la misma tasa de cambio para estimar 2019-2023
estimacion_2019 = registros_2024 / (1 + tasa_cambio_promedio)
estimacion_2020 = estimacion_2019 / (1 + tasa_cambio_promedio)
estimacion_2021 = estimacion_2020 / (1 + tasa_cambio_promedio)
estimacion_2022 = estimacion_2021 / (1 + tasa_cambio_promedio)
estimacion_2023 = estimacion_2022 / (1 + tasa_cambio_promedio)

print("Estimación de registros en 2019:", round(estimacion_2019))
print("Estimación de registros en 2020:", round(estimacion_2020))
print("Estimación de registros en 2021:", round(estimacion_2021))
print("Estimación de registros en 2022:", round(estimacion_2022))
print("Estimación de registros en 2023:", round(estimacion_2023))

Estimación de registros en 2019: 633792846
Estimación de registros en 2020: 633480749
Estimación de registros en 2021: 633168807
Estimación de registros en 2022: 632857017
Estimación de registros en 2023: 632545382


In [35]:
total_todos = total_todos_pachuca.copy()

In [36]:
# Remover comas y convertir la columna 'montos' a enteros# Ahora intenta realizar la sustitución
total_todos.loc[total_todos['año'] == '2019', 'montos'] = 633792846
total_todos.loc[total_todos['año'] == '2020', 'montos'] = 633480749
total_todos.loc[total_todos['año'] == '2021', 'montos'] = 633168807
total_todos.loc[total_todos['año'] == '2022', 'montos'] = 632857017
total_todos.loc[total_todos['año'] == '2023', 'montos'] = 632545382
# Verifica el cambio en el DataFrame
print(total_todos)

    año     montos         modalidad
0  2024  634105096  Créditos (todos)
1  2025  634417577  Créditos (todos)
2  2026  634730058  Créditos (todos)
3  2019  633792846  Créditos (todos)
4  2020  633480749  Créditos (todos)
5  2021  633168807  Créditos (todos)
6  2022  632857017  Créditos (todos)
7  2023  632545382  Créditos (todos)


In [21]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].mean().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, suma_creditos_año], ignore_index=True)

total_conteo_creditos

,modalidad,montos,año
0,1,8.313999e+05,2019
1,2,3.083739e+04,2019
2,3,7.619448e+05,2019
3,4,1.046279e+06,2019
4,1,1.111703e+06,2020
5,2,1.663406e+04,2020
6,3,7.781404e+05,2020
7,4,8.892310e+05,2020
8,1,1.044402e+06,2021
9,2,1.503075e+04,2021


In [22]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].mean().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3              4             5   \
 año                                                                           
 2019  9.748214e+05  8.432033e+05  7.019517e+05  747503.219000  9.598859e+05   
 2020  1.263388e+06  8.441523e+05  6.558994e+05       0.000000  8.480661e+05   
 2021  1.436877e+06  8.747464e+05  1.614926e+06  907570.806667  7.228699e+05   
 2022  7.877320e+05  8.661614e+05  1.392163e+06  648921.983333  1.168991e+06   
 2023  1.470818e+06  8.632339e+05  1.033721e+06  936952.735000  9.506788e+05   
 2024  1.066938e+06  1.668327e+06  1.817580e+05  887935.770000  1.029741e+06   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  7.684524e+05  5.977829e+05  8.403989e+05  7.774060e+05  8.942770e+05   
 2020  9.556773e+05  1.011581e+06  2.018581e+06  1.578504e+06  9.800427e+05   
 2021  7.527225e+05  8.010798e+05  9.63121

In [23]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]
modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13044\3999403270.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].mean().reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo['montos'] = nueva_conteo['montos'].map(format_number)

# Mostrar el DataFrame resultante
print(nueva_conteo)

    año modalidad   montos
0  2019     Nueva   820550
1  2020     Nueva  1062774
2  2021     Nueva   992657
3  2022     Nueva   974340
4  2023     Nueva  1120929
5  2024     Nueva  1070169
6  2025     Nueva  1145286
7  2026     Nueva  1184798


In [25]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [26]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo = mejoramiento_conteo.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo['montos'] = mejoramiento_conteo['montos'].map(format_number)
mejoramiento_conteo

,año,montos,modalidad
0,2019,750362,Usada
1,2020,804061,Usada
2,2021,860230,Usada
3,2022,942708,Usada
4,2023,1006835,Usada
5,2024,1000124,Usada
6,2025,1073782,Usada
7,2026,1125014,Usada


In [37]:
conteo_pachuca = pd.concat([
    total_todos,
    nueva_conteo,
    mejoramiento_conteo,
])

In [38]:
conteo_pachuca = conteo_pachuca.round(2)
conteo_pachuca

,año,montos,modalidad
0,2024,634105096,Créditos (todos)
1,2025,634417577,Créditos (todos)
2,2026,634730058,Créditos (todos)
3,2019,633792846,Créditos (todos)
4,2020,633480749,Créditos (todos)
5,2021,633168807,Créditos (todos)
6,2022,632857017,Créditos (todos)
7,2023,632545382,Créditos (todos)
0,2019,820550,Nueva
1,2020,1062774,Nueva


In [39]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año     montos         modalidad
3  2019  633792846  Créditos (todos)
0  2019     820550             Nueva
0  2019     750362             Usada
1  2020    1062774             Nueva
1  2020     804061             Usada
4  2020  633480749  Créditos (todos)
5  2021  633168807  Créditos (todos)
2  2021     992657             Nueva
2  2021     860230             Usada
3  2022     942708             Usada
3  2022     974340             Nueva
6  2022  632857017  Créditos (todos)
4  2023    1120929             Nueva
7  2023  632545382  Créditos (todos)
4  2023    1006835             Usada
0  2024  634105096  Créditos (todos)
5  2024    1070169             Nueva
5  2024    1000124             Usada
1  2025  634417577  Créditos (todos)
6  2025    1145286             Nueva
6  2025    1073782             Usada
2  2026  634730058  Créditos (todos)
7  2026    1184798             Nueva
7  2026    1125014             Usada


In [40]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto promedio anual",
    yaxis_tickformat=",.0f",  # Formatear el eje y con separadores de coma y sin decimales
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_promedio', carpeta='assets/graficas')
fig.show()

# JJSJSJD

# **3.4 Forecast Monto de créditos generales. [PROMEDIO]**

In [2]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).mean().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos = pd.concat(dfs, ignore_index=True)
total_suma_creditos

,año,mes,montos
0,2019,1,7.367233e+05
1,2019,2,5.900232e+05
2,2019,3,5.086947e+05
3,2019,4,4.374097e+05
4,2019,5,6.915274e+05
...,...,...,...
62,2024,3,3.749487e+05
63,2024,4,1.024158e+06
64,2024,5,7.829447e+05
65,2024,6,8.110442e+05


In [3]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].mean().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, suma_creditos_año], ignore_index=True)

total_conteo_creditos

,modalidad,montos,año
0,1,8.313999e+05,2019
1,2,3.083739e+04,2019
2,3,7.619448e+05,2019
3,4,1.046279e+06,2019
4,1,1.111703e+06,2020
5,2,1.663406e+04,2020
6,3,7.781404e+05,2020
7,4,8.892310e+05,2020
8,1,1.044402e+06,2021
9,2,1.503075e+04,2021


In [4]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].mean().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3              4             5   \
 año                                                                           
 2019  9.748214e+05  8.432033e+05  7.019517e+05  747503.219000  9.598859e+05   
 2020  1.263388e+06  8.441523e+05  6.558994e+05       0.000000  8.480661e+05   
 2021  1.436877e+06  8.747464e+05  1.614926e+06  907570.806667  7.228699e+05   
 2022  7.877320e+05  8.661614e+05  1.392163e+06  648921.983333  1.168991e+06   
 2023  1.470818e+06  8.632339e+05  1.033721e+06  936952.735000  9.506788e+05   
 2024  1.066938e+06  1.668327e+06  1.817580e+05  887935.770000  1.029741e+06   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  7.684524e+05  5.977829e+05  8.403989e+05  7.774060e+05  8.942770e+05   
 2020  9.556773e+05  1.011581e+06  2.018581e+06  1.578504e+06  9.800427e+05   
 2021  7.527225e+05  8.010798e+05  9.63121

In [5]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]
modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_2868\3999403270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = modalidad1[['año', 'mes']]
y = modalidad1['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [7]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].mean().reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo['montos'] = nueva_conteo['montos'].map(format_number)

# Mostrar el DataFrame resultante
print(nueva_conteo)

    año modalidad   montos
0  2019     Nueva   820550
1  2020     Nueva  1062774
2  2021     Nueva   992657
3  2022     Nueva   974340
4  2023     Nueva  1120929
5  2024     Nueva  1070169
6  2025     Nueva  1145286
7  2026     Nueva  1184798


In [8]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [9]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo = mejoramiento_conteo.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo['montos'] = mejoramiento_conteo['montos'].map(format_number)
mejoramiento_conteo

,año,montos,modalidad
0,2019,750362,Usada
1,2020,804061,Usada
2,2021,860230,Usada
3,2022,942708,Usada
4,2023,1006835,Usada
5,2024,1000124,Usada
6,2025,1073782,Usada
7,2026,1125014,Usada


In [10]:
conteo_pachuca = pd.concat([
    #total_todos,
    nueva_conteo,
    mejoramiento_conteo,
])

In [11]:
conteo_pachuca = conteo_pachuca.round(2)
conteo_pachuca

,año,modalidad,montos
0,2019,Nueva,820550
1,2020,Nueva,1062774
2,2021,Nueva,992657
3,2022,Nueva,974340
4,2023,Nueva,1120929
5,2024,Nueva,1070169
6,2025,Nueva,1145286
7,2026,Nueva,1184798
0,2019,Usada,750362
1,2020,Usada,804061


In [12]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año modalidad   montos
0  2019     Nueva   820550
0  2019     Usada   750362
1  2020     Usada   804061
1  2020     Nueva  1062774
2  2021     Nueva   992657
2  2021     Usada   860230
3  2022     Usada   942708
3  2022     Nueva   974340
4  2023     Usada  1006835
4  2023     Nueva  1120929
5  2024     Nueva  1070169
5  2024     Usada  1000124
6  2025     Usada  1073782
6  2025     Nueva  1145286
7  2026     Nueva  1184798
7  2026     Usada  1125014


In [13]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
modalidades = conteo_pachuca['modalidad'].unique()
#colores = ['red', 'lightgreen']  # Definir los colores a usar
for i, modalidad in enumerate(modalidades):
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center",  # Posicionar el texto encima de los puntos
        #line=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
        #marker=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto promedio anual",
    yaxis_tickformat=",.0f",  # Formatear el eje y con separadores de coma y sin decimales
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_promedio', carpeta='assets/graficas')
fig.show()